<a href="https://colab.research.google.com/github/gkbrasselle/DS4002Project1/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'amazon-fine-food-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F18%2F2157%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240919%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240919T233448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9b447db312786406ae4c63e3313b2164dd1abe604ea295950689e24237e27b5170802e91e753392d272b894f12e94b19bf5d59660267312610249298f22c09dc91057b05870230e8c013110a522af6ba0ff2e87b32242e0f8454bf73941a06b1b0dedfeab2f25567354eb0c54e31ee1d7fa3d4b942fb79894d0e903d37519d4b78439bfd99f9b6d6952d99f380e0b59f97e2189ff63f3bd6db427e2f849c8f6c9a48ef016b4b6b247a01b83b9ca2dd3e8668db36069337e57babff58cfaaff14b6e45e392b35f1ea781afae5b13c19240796141f2a4cdfda98fde03f8249bb2f8a8220fa01e3bbf26ef2421ccb17fb9672f3b4dc9282560c94aef6a6646e95d3'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 253873708 bytes downloaded
Downloaded and uncompressed: amazon-fine-food-reviews
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/database.sqlite


In [3]:
# Load Data & store it in a dataframe
df = pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
# Identify unnecessary columns and remove them from the data frame
drop_list = ["Id", "UserId", "ProfileName", "Time"]
clean_df = df.drop(drop_list, axis=1)
clean_df.head()


,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text
0,B001E4KFG0,1,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,0,0,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,1,1,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,3,3,2,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,0,0,5,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
print(clean_df.shape, '\n')

(568454, 6) 



In [6]:
# We are going to evaluate the percentage of users who found a review helpful
# Don't need any rows where there weren't any users who indicated if a review was helpful or unhelpful
filtered_df = clean_df[clean_df['HelpfulnessDenominator']!= 0]
filtered_df.head(15)

,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text
0,B001E4KFG0,1,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,B000LQOCH0,1,1,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,3,3,2,Cough Medicine,If you are looking for the secret ingredient i...
8,B000E7L2R4,1,1,5,Yay Barley,Right now I'm mostly just sprouting this so my...
10,B0001PB9FE,1,1,5,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...
11,B0009XLVG0,4,4,5,"My cats LOVE this ""diet"" food better than thei...",One of my boys needed to lose some weight and ...
12,B0009XLVG0,1,1,1,My Cats Are Not Fans of the New Food,My cats have been happily eating Felidae Plati...
13,B001GVISJM,2,2,4,fresh and greasy!,good flavor! these came securely packed... the...
14,B001GVISJM,4,5,5,Strawberry Twizzlers - Yummy,The Strawberry Twizzlers are my guilty pleasur...
15,B001GVISJM,4,5,5,"Lots of twizzlers, just what you expect.",My daughter loves twizzlers and this shipment ...


In [7]:
print(filtered_df.shape, '\n')
print(filtered_df.dtypes, '\n')

#even when we filter out the reviews that had no indicator of helpfulness, we still have 298,402 rows to work with

(298402, 6) 

ProductId                 object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Summary                   object
Text                      object
dtype: object 



In [8]:
# Calculate the ratio of people who found a review helpful
filtered_df["HelpfulnessRatio"] = filtered_df["HelpfulnessNumerator"]/filtered_df["HelpfulnessDenominator"]


<ipython-input-8-c121f2473b9b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["HelpfulnessRatio"] = filtered_df["HelpfulnessNumerator"]/filtered_df["HelpfulnessDenominator"]


In [9]:
# Make sure that the new column was created correctly
filtered_df.head()
cols = ['ProductId', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'HelpfulnessRatio', 'Score', 'Summary', 'Text']

In [10]:
# rearrange the columns
filtered_df = filtered_df.loc[:, cols]

In [11]:
filtered_df.head()

,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,HelpfulnessRatio,Score,Summary,Text
0,B001E4KFG0,1,1,1.0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,B000LQOCH0,1,1,1.0,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,3,3,1.0,2,Cough Medicine,If you are looking for the secret ingredient i...
8,B000E7L2R4,1,1,1.0,5,Yay Barley,Right now I'm mostly just sprouting this so my...
10,B0001PB9FE,1,1,1.0,5,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...


In [12]:
# Create a new column that will contain the word count of the review
filtered_df["WordCount"] = filtered_df["Text"].str.split().str.len()
filtered_df.head()

,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,HelpfulnessRatio,Score,Summary,Text,WordCount
0,B001E4KFG0,1,1,1.0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,48
2,B000LQOCH0,1,1,1.0,4,"""Delight"" says it all",This is a confection that has been around a fe...,94
3,B000UA0QIQ,3,3,1.0,2,Cough Medicine,If you are looking for the secret ingredient i...,41
8,B000E7L2R4,1,1,1.0,5,Yay Barley,Right now I'm mostly just sprouting this so my...,26
10,B0001PB9FE,1,1,1.0,5,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...,146


In [13]:
filtered_df.to_csv('cleaned.csv')